# 02. 클러스터링 분석

## 목적
- 5단계 상태 라벨링 (정상영업, D-12m, D-9m, D-6m, D-3m)
- K-means 클러스터링으로 가맹점 세분화
- 클러스터별 임박비중 계산 (위험도 평가)

## 1. 데이터 로드 및 날짜 처리

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
import warnings
warnings.filterwarnings('ignore')

data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv"
df = pd.read_csv(data_path)

# 날짜 변환
df['기준년월_dt'] = pd.to_datetime(df['기준년월'].astype(str), format='%Y%m', errors='coerce')

# 폐업월 계산
def _to_periodM_from_any(s):
    s = s.astype(str).str.replace(r'\.0$', '', regex=True).str.strip()
    dt = pd.to_datetime(s, errors='coerce')
    return dt.dt.to_period('M')

df['_관측월'] = df['기준년월_dt'].dt.to_period('M')
df['_폐업월'] = _to_periodM_from_any(df['폐업일'])

# 개월차 계산
y1, m1 = df['_폐업월'].dt.year, df['_폐업월'].dt.month
y0, m0 = df['_관측월'].dt.year, df['_관측월'].dt.month
df['_months_to_close'] = (y1 - y0) * 12 + (m1 - m0)

print(f"데이터 형태: {df.shape}")
print(f"폐업 가맹점: {df['_폐업월'].notna().sum():,}개")

데이터 형태: (86263, 192)
폐업 가맹점: 2,334개


## 2. 5단계 상태 라벨링

In [2]:
# 정상영업: 한번도 폐업 기록 없는 가맹점의 최신 데이터
closed_any = df.groupby('가맹점구분번호')['_폐업월'].transform(lambda s: s.notna().any())
normal_latest = (df[~closed_any]
                 .sort_values(['가맹점구분번호', '기준년월_dt'])
                 .groupby('가맹점구분번호', as_index=False)
                 .tail(1)
                 .assign(상태='정상영업'))

print(f"정상영업: {len(normal_latest):,}개")

# 폐업 D-k: 각 가맹점에서 정확히 D-k 개월 전 스냅샷 1건
def _pick_preclose(k):
    sub = df[df['_months_to_close'].eq(k)].copy()
    if sub.empty:
        return sub.assign(상태=f'D-{k}m')
    sub = (sub.sort_values(['가맹점구분번호', '기준년월_dt'])
              .groupby('가맹점구분번호', as_index=False)
              .tail(1))
    sub['상태'] = f'D-{k}m'
    return sub

d12 = _pick_preclose(12)
d9 = _pick_preclose(9)
d6 = _pick_preclose(6)
d3 = _pick_preclose(3)

print(f"D-12m: {len(d12):,}개")
print(f"D-9m: {len(d9):,}개")
print(f"D-6m: {len(d6):,}개")
print(f"D-3m: {len(d3):,}개")

# 통합
ana = pd.concat([normal_latest, d12, d9, d6, d3], ignore_index=True)

STATUS_ORDER = ["정상영업", "D-12m", "D-9m", "D-6m", "D-3m"]
STATUS_RANK = {s: i for i, s in enumerate(STATUS_ORDER)}
ana['_상태_ord'] = ana['상태'].map(STATUS_RANK)

print(f"\n통합 데이터: {len(ana):,}개")
print("\n[상태 분포]:")
print(ana['상태'].value_counts().sort_index())

정상영업: 4,043개
D-12m: 105개
D-9m: 113개
D-6m: 32개
D-3m: 32개

통합 데이터: 4,325개

[상태 분포]:
상태
D-12m     105
D-3m       32
D-6m       32
D-9m      113
정상영업     4043
Name: count, dtype: int64


### 해석
- 정상영업: 폐업 이력이 없는 가맹점의 최근 스냅샷
- D-k: 폐업 k개월 전 시점의 데이터
- 시간 기반 라벨링으로 조기 경보 가능

## 3. K-means 클러스터링

In [3]:
# 변수 선택 (결측 50% 기준)
exclude_cols = {
    '가맹점구분번호', '기준년월', '기준연월', '기준분기', '개설일', '폐업일',
    '상권_코드', '상권_코드_명', '구분지역', '지역명',
    '업종', '상태', '_상태_ord', '기준년월_dt',
    '_폐업월', '_관측월', '_months_to_close'
}

cand_cols = [c for c in ana.columns if c not in exclude_cols]
feat_cols = [c for c in cand_cols if pd.api.types.is_numeric_dtype(ana[c]) and ana[c].notna().mean() > 0.5]

print(f"선택된 변수 수: {len(feat_cols)}개")

# 결측치 처리
cluster_df = ana[['가맹점구분번호', '상태', '_상태_ord'] + feat_cols].copy()
for c in feat_cols:
    if cluster_df[c].isna().any():
        cluster_df[c] = cluster_df[c].fillna(cluster_df[c].median())

# 스케일링
X = cluster_df[feat_cols].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"변수 행렬 형태: {X.shape}")

선택된 변수 수: 169개
변수 행렬 형태: (4325, 169)


In [4]:
# k 선택 (k=3~7)
k_range = range(3, 8)
evaluation_results = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init=30, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    silhouette = silhouette_score(X_scaled, labels)
    db_score = davies_bouldin_score(X_scaled, labels)
    evaluation_results.append({
        'k': k,
        'silhouette': silhouette,
        'davies_bouldin': db_score
    })
    print(f"k={k}: Silhouette={silhouette:.4f}, DB={db_score:.4f}")

eval_df = pd.DataFrame(evaluation_results)
best_k = int(eval_df.loc[eval_df['silhouette'].idxmax(), 'k'])
print(f"\n최적 k: {best_k}")

# 최종 클러스터링
kmeans_final = KMeans(n_clusters=best_k, n_init=30, random_state=42)
cluster_df['cluster'] = kmeans_final.fit_predict(X_scaled)
print(f"클러스터 할당 완료")

k=3: Silhouette=0.5882, DB=1.1743
k=4: Silhouette=0.5725, DB=1.1266
k=5: Silhouette=0.2783, DB=1.3922
k=6: Silhouette=0.1542, DB=1.5429
k=7: Silhouette=0.1841, DB=1.3991

최적 k: 3
클러스터 할당 완료


### 통계적 해석
- Silhouette Score: 클러스터 간 분리도 (높을수록 좋음)
- Davies-Bouldin Index: 클러스터 내 응집도 (낮을수록 좋음)
- 최적 k 선택: Silhouette 최대화

## 4. 임박비중 계산

In [5]:
# 클러스터별 상태 분포
cluster_risk = cluster_df.groupby('cluster').agg(
    가맹점수=('가맹점구분번호', 'count'),
    정상=('상태', lambda s: (s == '정상영업').sum()),
    D12=('상태', lambda s: (s == 'D-12m').sum()),
    D9=('상태', lambda s: (s == 'D-9m').sum()),
    D6=('상태', lambda s: (s == 'D-6m').sum()),
    D3=('상태', lambda s: (s == 'D-3m').sum())
)

# 임박비중 계산
cluster_risk['임박비중'] = (
    cluster_risk['D12'] + cluster_risk['D9'] + 
    cluster_risk['D6'] + cluster_risk['D3']
) / cluster_risk['가맹점수']

cluster_risk = cluster_risk.sort_values('임박비중', ascending=False)

print("\n클러스터별 임박비중:")
print(cluster_risk[['가맹점수', '정상', 'D12', 'D9', 'D6', 'D3', '임박비중']])

# 고위험 클러스터
high_risk_clusters = cluster_risk[cluster_risk['임박비중'] > 0.05].index.tolist()
print(f"\n고위험 클러스터: {high_risk_clusters}")


클러스터별 임박비중:
         가맹점수    정상  D12  D9  D6  D3      임박비중
cluster                                       
1          61     0   14  16  16  15  1.000000
2         183     0   91  63  14  15  1.000000
0        4081  4043    0  34   2   2  0.009311

고위험 클러스터: [1, 2]


<cell_type>markdown</cell_type>## 5. 결과 저장

<cell_type>markdown</cell_type>### 정책적 의미
- 임박비중 > 5%: 고위험 클러스터 → 즉시 개입 필요
- 임박비중 2~5%: 중위험 → 집중 관리
- 클러스터별 맞춤 정책 수립 가능

In [6]:
import os
output_dir = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석"
os.makedirs(output_dir, exist_ok=True)

# 클러스터 할당 결과
result_df = cluster_df[['가맹점구분번호', 'cluster', '상태', '_상태_ord']].copy()
result_df.to_csv(f"{output_dir}/클러스터링_결과_완전판.csv", index=False, encoding='utf-8-sig')

# 임박비중 통계
cluster_risk.to_csv(f"{output_dir}/클러스터_임박비중_완전판.csv", encoding='utf-8-sig')

print("결과 저장 완료")

결과 저장 완료


## 종합 결론

### 클러스터링 성과
1. **최적 클러스터 수**: Silhouette Score 기준으로 결정
2. **고위험 클러스터 식별**: 임박비중 > 5% 기준
3. **정책 우선순위**: 고위험 클러스터 집중 지원

### 다음 단계
- 공간분석 (LISA) 추가
- 지도 시각화로 위험 지역 파악